In [1]:
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper


In [2]:
pip install wikipedia

Note: you may need to restart the kernel to use updated packages.


In [3]:
wiki_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=250)
wiki=WikipediaQueryRun(api_wrapper=wiki_wrapper)



In [4]:
#creaated the 1 tool
wiki.name


'wikipedia'

In [5]:
pip install arxiv


Note: you may need to restart the kernel to use updated packages.


In [6]:
#Same for the arxiv tool

arxvi_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=250)
arxiv=ArxivQueryRun(api_erapper=arxvi_wrapper)

In [7]:
#develop the 2nd inbuilt tool
arxiv.name

'arxiv'

In [43]:
# developing the custome tool with RAG
#1 load the API key from the .env file
#2 Authenticate with the huggingface API key

import os 
from dotenv import  load_dotenv

load_dotenv()
API_KEY=os.getenv("HF_API_KEY")
 

In [9]:
#login to the huggingface hub using the API key
from huggingface_hub import login

login(token=API_KEY)

c:\Users\Nikhil Warge\OneDrive\Documents\GitHub\Implementing-ANN-and-RNN-\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
#start developting the custome tool with RAG
#load the data from the web using webbase loader
from langchain_community.document_loaders import WebBaseLoader
#vectore store to store the data 
from langchain_community.vectorstores import faiss
#load the embeding technique from the huggingface hub
from langchain.embeddings import HuggingFaceInferenceAPIEmbeddings
#for converting the text data into corpu or chunk 
from langchain.text_splitter import  RecursiveCharacterTextSplitter


USER_AGENT environment variable not set, consider setting it to identify your requests.


In [11]:
#loading embeding model from the huggingfface hub
HF_APIKEY=os.getenv("HF_API_KEY")
embidder=HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_APIKEY,
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    
)

In [12]:
#store the web data into the vector store using faiss
loader=WebBaseLoader("https://www.analyticsvidhya.com/blog/2024/11/generative-ai-interview-questions/")
#load the data from the web using webbase loader
document=loader.load()

#2 split the data into chunks using RecursiveCharacterTextSplitter

text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=540,
    chunk_overlap=0
    )
doc=text_splitter.split_documents(document)

#3 Initialise the hhuggingface inferance API embedding model

embedding=HuggingFaceInferenceAPIEmbeddings(
    api_key=HF_APIKEY,
    model_name="sentence-transformers/all-MiniLM-L6-v2")
#step 4 create the vectore store using FAISS

vectorestore=faiss.FAISS.from_documents(doc,embedding)

#step 5 create the retriever using the vectore store
retriever=vectorestore.as_retriever(search_kwargs={"k":1})
#step 6 create the custome tool uaing the retriever and the embeding model

from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"interview_questions","search any information about any question")

retriever_tool.name

'interview_questions'

In [13]:
#now chain the tools together using the langchain agent

tool=[wiki,arxiv,retriever_tool]

tool

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from 'c:\\Users\\Nikhil Warge\\OneDrive\\Documents\\GitHub\\Implementing-ANN-and-RNN-\\venv\\Lib\\site-packages\\wikipedia\\__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=250)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=3, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=4000)),
 Tool(name='interview_questions', description='search any information about any question', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x0000021F94503100>, retriever=VectorStoreRetriever(tags=['FAISS', 'HuggingFaceInferenceAPIEmbeddings'], vectorstore=<lang

In [34]:
#now rull all the tools with agents and the LLM model

##Tools , LLM >>Agent Executer
from langchain_groq import ChatGroq
from dotenv import  load_dotenv
import os
load_dotenv()

groq_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=groq_key,model="llama-3.3-70b-versatile",temperature=0.1)
llm


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000021FDDDD27D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000021FDDDCC090>, model_name='llama-3.3-70b-versatile', temperature=0.1, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [15]:
pip install langchain-openai


Note: you may need to restart the kernel to use updated packages.


In [35]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

c:\Users\Nikhil Warge\OneDrive\Documents\GitHub\Implementing-ANN-and-RNN-\venv\Lib\site-packages\langsmith\client.py:253: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


In [36]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [37]:
#define the AGENTS

from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tool,prompt)
agent

RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages

In [41]:
#define agent executor

from langchain.agents import AgentExecutor
agent_executer=AgentExecutor(agent=agent,tools=tool,verbose=True)
agent_executer.invoke({"input":"Shivaji Maharaj"})




> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Shivaji Maharaj'}`


Page: Shivaji
Summary: Shivaji I (Shivaji Shahaji Bhonsale, Marathi pronunciation: [ʃiˈʋaːdʑiː ˈbʱos(ə)le]; c. 19 February 1630 – 3 April 1680) was an Indian ruler and a member of the Bhonsle dynasty. Shivaji carved out his own independent kingdom frShivaji Maharaj, also known as Chhatrapati Shivaji, was a Indian ruler and a member of the Bhonsle dynasty. He was born on February 19, 1630, and died on April 3, 1680. Shivaji is known for carving out his own independent kingdom from the Mughal Empire and establishing the Maratha Empire. He is considered one of the greatest rulers in Indian history and is revered as a hero and a symbol of Indian nationalism.

> Finished chain.


{'input': 'Shivaji Maharaj',
 'output': 'Shivaji Maharaj, also known as Chhatrapati Shivaji, was a Indian ruler and a member of the Bhonsle dynasty. He was born on February 19, 1630, and died on April 3, 1680. Shivaji is known for carving out his own independent kingdom from the Mughal Empire and establishing the Maratha Empire. He is considered one of the greatest rulers in Indian history and is revered as a hero and a symbol of Indian nationalism.'}

In [42]:
agent_executer.invoke({"input":"Tell me how to evaluate the performance of the Gen AI model?"})




> Entering new AgentExecutor chain...

Invoking: `interview_questions` with `{'query': 'Evaluating the performance of the Gen AI model'}`


Learn everything there is to know about generative AI and become a GenAI expert with our GenAI Pinnacle Program.
Invoking: `wikipedia` with `{'query': 'Evaluating the performance of the Gen AI model'}`


Page: Gemini (language model)
Summary: Gemini is a family of multimodal large language models developed by Google DeepMind, and the successor to LaMDA and PaLM 2. Comprising Gemini Ultra, Gemini Pro, Gemini Flash, and Gemini Nano, it was announced on
Invoking: `arxiv` with `{'query': 'Evaluating the performance of the Gen AI model'}`


Published: 2025-02-12
Title: Cognify: Supercharging Gen-AI Workflows With Hierarchical Autotuning
Authors: Zijian He, Reyna Abhyankar, Vikranth Srivatsa, Yiying Zhang
Summary: Today's gen-AI workflows that involve multiple ML model calls, tool/API
calls, data retrieval, or generic code execution are often tuned man

{'input': 'Tell me how to evaluate the performance of the Gen AI model?',
 'output': "Evaluating the performance of the Gen AI model can be done by assessing its ability to generate high-quality content, such as text, images, or videos, and its ability to complete tasks efficiently. The performance of Gen AI can be evaluated using various metrics, such as answer quality scores, task completion time, and user satisfaction.\n\nIn a field experiment, researchers found that Gen AI improved answer quality scores by 17% and task completion time by 34% for a document understanding task, but had mixed results for a data analysis task. Another study explored the use of Gen AI in content creation on YouTube and found that creators used Gen AI tools to produce imaginative images, AI-generated videos, and articles.\n\nTo evaluate the performance of Gen AI, researchers and developers can use various methods, such as:\n\n1. Human evaluation: Human evaluators can assess the quality of the generated c